<a href="https://colab.research.google.com/github/noelmathen/RSMS-Feedback-Form-Filling-Automation/blob/main/Feedback_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install selenium pandas

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [808 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.7 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,691 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://

In [6]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time

def select_feedback_options(driver):
    for question_number in range(1, 18):
        try:
            radio_name = f"{10904 + question_number}"
            if question_number in range(1, 16):
                driver.find_element(By.CSS_SELECTOR, f"input[name='{radio_name}'][value='4']").click()
            elif question_number == 16 or question_number == 17:
                text_area_name = f"{10904 + question_number}"
                driver.find_element(By.CSS_SELECTOR, f"textarea[name='{text_area_name}']").send_keys('-')
            time.sleep(0.01)
        except NoSuchElementException:
            print(f"Element for question {question_number} not found.")

    try:
        driver.find_element(By.NAME, "B1").click()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "Subject_Code")))
        time.sleep(2)
    except NoSuchElementException:
        print("Button or Subject Code input not found.")

username = input("Enter your uid: ")
password = input("Enter your password: ")

try:
    driver = web_driver()
    driver.maximize_window()
    driver.get("https://www.rajagiritech.ac.in/stud/ktu/student/")
    driver.find_element(By.NAME, "Userid").send_keys(username)
    driver.find_element(By.NAME, "Password").send_keys(password)
    driver.find_element(By.XPATH, "//input[@type='submit']").click()
    driver.find_element(By.LINK_TEXT, "2024 Mid Semester Feedback Even").click()
    time.sleep(2)

    while True:
        try:
            dropdown = Select(driver.find_element(By.NAME, "Subject_Code"))
            first_option = dropdown.first_selected_option.text
            print(f"Feedback for {first_option} completed")
            dropdown.select_by_visible_text(first_option)
            select_feedback_options(driver)
        except NoSuchElementException:
            print("Feedback submission completed!.")
            break
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            break
finally:
    driver.quit()
